In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report


In [4]:
# Load the dataset
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
data = load_dataset("marianeft/heart-diease-dataset")
print(data)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

e:\setups\anaconda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asus\.cache\huggingface\hub\datasets--marianeft--heart-diease-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


heart.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1025 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
        num_rows: 1025
    })
})


In [6]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("marianeft/heart-diease-dataset")

# Convert the 'train' split to a DataFrame
df = dataset['train'].to_pandas()


In [7]:
print(df.dtypes)


age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object


In [8]:

le = LabelEncoder()

columns_to_encode = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

for col in columns_to_encode:
    if df[col].dtype == 'object' or df[col].dtype.name == 'category':
        df[col] = le.fit_transform(df[col])


In [10]:
X = df.drop(columns='target')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Train a Decision Tree to get feature importance
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Get feature importance scores
feature_importances = dt.feature_importances_

# Create a DataFrame to view importance rankings
feature_names = X.columns  # Assuming X is a Pandas DataFrame
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort and select the top 7 features
top_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(7)['Feature'].values

print("Top 7 Selected Features:", top_features)

# Keep only the selected features in training and testing sets
X_train_selected = pd.DataFrame(X_train, columns=feature_names)[top_features]
X_test_selected = pd.DataFrame(X_test, columns=feature_names)[top_features]

# Retrain the model with selected features
dt_selected = DecisionTreeClassifier(random_state=42)
dt_selected.fit(X_train_selected, y_train)

# Evaluate with cross-validation
cv_scores = cross_val_score(dt_selected, X_train_selected, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean CV Accuracy with Top 7 Features: {cv_scores.mean():.4f}")

# Predict and print classification report
y_pred = dt_selected.predict(X_test_selected)
print("Classification Report on Test Data:")
print(classification_report(y_test, y_pred))

Top 7 Selected Features: ['cp' 'thal' 'ca' 'chol' 'oldpeak' 'age' 'thalach']
Cross-Validation Accuracy Scores: [0.98780488 0.9695122  0.98780488 0.98780488 0.93292683]
Mean CV Accuracy with Top 7 Features: 0.9732
Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       102
           1       1.00      0.97      0.99       103

    accuracy                           0.99       205
   macro avg       0.99      0.99      0.99       205
weighted avg       0.99      0.99      0.99       205



In [12]:
import pickle

with open("heart_model.pkl", "wb") as file:
    pickle.dump(dt_selected, file)

**How to use ?**

In [13]:
import pickle
import numpy as np

# ['cp' 'thal' 'ca' 'chol' 'oldpeak' 'age' 'thalach']
"""
age:	Age of the patient (years)	Older individuals have a higher risk of heart disease.
cp (Chest Pain Type):	0 = Typical Angina, 1 = Atypical Angina, 2 = Non-anginal Pain, 3 = Asymptomatic	Certain types of chest pain are strong indicators of heart problems.
chol (Serum Cholesterol):	Cholesterol level in mg/dL	High cholesterol can lead to plaque buildup in arteries, increasing heart disease risk.
thalach (Maximum Heart Rate Achieved):	Maximum recorded heart rate during stress testing	A lower max heart rate could indicate heart disease.
oldpeak (ST Depression):	ST segment depression in ECG	Higher values may indicate ischemia (reduced blood flow to the heart).
ca (Number of Major Vessels Colored by Fluoroscopy)	0–3 :	Higher values indicate blocked arteries, increasing heart disease risk.
thal (Thalassemia) :	0 = Normal, 1 = Fixed Defect, 2 = Reversible Defect	Blood disorder that can affect oxygen transport and heart function.

"""

# Load the model
with open("heart_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

input_data = np.array([[1, 2, 0, 210, 0.7, 34, 192]])  

# Make a prediction
y_pred = loaded_model.predict(input_data)

print("Predicted Output:", y_pred)

Predicted Output: [1]


e:\setups\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


to Compute Old Peak using the ecg values

If you use a 12-bit ADC (e.g., ESP32, STM32):

The ADC range is 0 to 4095.
Reference voltage (Vref) is typically 3.3V or 5V.
The conversion formula:
𝑉
analog
=
(
ADC Value
×
𝑉
ref
4095
)
V 
analog
​
 =( 
4095
ADC Value×V 
ref
​
 
​
 )
Example: If ADC reads 2048 and Vref is 3.3V,
𝑉
analog
=
(
2048
×
3.3
𝑉
4095
)
=
1.65
𝑉
V 
analog
​
 =( 
4095
2048×3.3V
​
 )=1.65V

In [16]:
def predict_heart_disease(model, input_data):
    input_data = np.array(input_data).reshape(1, -1)
    if input_data.shape[1] != 7:
        raise ValueError("Expected input with 7 features: [cp, thal, ca, chol, oldpeak, age, thalach]")
    return model.predict(input_data)[0]

In [17]:
import joblib
joblib.dump(dt_selected, "heart_model.joblib")

['heart_model.joblib']

In [18]:
model = joblib.load("heart_model.joblib")